# Convolutional Neural Network (CNN)

I am using a CNN written in `tensorflow` and `keras`. CNN is a class of neural networks commonly applied to image data for image classification. In this technique, the image is divided into subgrids, and these subgrids are passed on the entire image. The passing of these subgrids is called convolution. In each convolution, the model is learning features that occupy that space at a higher level. In other words, in each convolution, the model passes the subgrids on the entire image and learns general features across the image.

In [1]:
# %%bash
# unzip /content/drive/MyDrive/TDI_Capstone_Project/partial_dataset.zipb

# Import libraries

In [2]:
%config Completer.use_jedi = False
import glob
import os
import cv2
from natsort import natsorted
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

print(f"Tensorflow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

Tensorflow version: 2.4.1
Keras version: 2.4.0


# Make labels

In [3]:
labels_images = pd.read_csv('partial_dataset/train/labels_images.csv')
labels_images.rename(columns={'0':'labels'}, inplace=True)
labels = labels_images['labels'].to_numpy() - 1  # to be in range(0,6)
labels

array([0, 0, 0, ..., 5, 5, 5])

# Prepare images

In [4]:
# 1 - get list of paths
list_images = [f"partial_dataset/train/images/{x}" for x in labels_images['filename'].to_list()]
list_images[:5]

['partial_dataset/train/images/18X67NLZ2_707-1603-1219-2115.jpg',
 'partial_dataset/train/images/18X67NLZ2_1219-579-1731-1091.jpg',
 'partial_dataset/train/images/18X67NLZ2_1219-1091-1731-1603.jpg',
 'partial_dataset/train/images/18X67NLZ2_1219-1603-1731-2115.jpg',
 'partial_dataset/train/images/26G8P9NJF_576-4531-1088-5043.jpg']

In [5]:
# 2 - function to read each image into np.array

def read_image(data_x):
    """Read obj with the path for images.
    data_x ==  *_x
    """

    W, H = 300, 300 # set final size

    x = cv2.imread(data_x, cv2.IMREAD_COLOR) # read in colors
    x = cv2.resize(x, (W, H)) # resize
    x = x / 255.0 # normalize values 0 to 1
    x = x.astype(np.float32) # make it np.float32
    
    return x

In [6]:
# 3 - get an array of images
images = np.array([read_image(x) for x in list_images])

In [7]:
# 4 - split labels and images into train and test
train_images, test_images = train_test_split(images, test_size=0.4, random_state=88)
train_labels, test_labels = train_test_split(labels, test_size=0.4, random_state=88)

print(
    f"Train images: {len(train_images)}, Train labels: {len(train_labels)}, Test images: {len(test_images)}, Test labels: {len(test_labels)}"
)

Train images: 2828, Train labels: 2828, Test images: 1886, Test labels: 1886


In [8]:
# check data type and shape
print(type(train_images), type(train_labels))
print(train_images.shape, train_labels.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(2828, 300, 300, 3) (2828,)


In [9]:
# check data type and shape
print(type(test_images), type(test_labels))
print(test_images.shape, test_labels.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(1886, 300, 300, 3) (1886,)


# Make a Convolutional Neural Network (CNN)

In [10]:
model = keras.Sequential([
        keras.layers.AveragePooling2D(pool_size=6, strides=6, input_shape=(300, 300, 3)),
        keras.layers.Conv2D(64, 3, activation='relu'),
        keras.layers.Conv2D(32, 2, activation='relu'),
        keras.layers.MaxPool2D(3, 3),
        keras.layers.Dropout(0.5),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10, batch_size=80)

acc = round(model.evaluate(test_images, test_labels, verbose=0)[1], 4)*100
print(f"Accuracy on test data is {acc}%")

Epoch 1/10
36/36 [==============================] - 21s 435ms/step - loss: 1.2422 - accuracy: 0.6002
Epoch 2/10
36/36 [==============================] - 11s 314ms/step - loss: 1.0602 - accuracy: 0.6632
Epoch 3/10
36/36 [==============================] - 11s 295ms/step - loss: 1.0041 - accuracy: 0.6663
Epoch 4/10
36/36 [==============================] - 11s 298ms/step - loss: 0.9228 - accuracy: 0.6761
Epoch 5/10
36/36 [==============================] - 12s 325ms/step - loss: 0.8843 - accuracy: 0.6790
Epoch 6/10
36/36 [==============================] - 11s 315ms/step - loss: 0.8621 - accuracy: 0.6865
Epoch 7/10
36/36 [==============================] - 11s 292ms/step - loss: 0.8226 - accuracy: 0.6950
Epoch 8/10
36/36 [==============================] - 10s 283ms/step - loss: 0.7541 - accuracy: 0.7239
Epoch 9/10
36/36 [==============================] - 10s 279ms/step - loss: 0.7527 - accuracy: 0.7474
Epoch 10/10
36/36 [==============================] - 11s 294ms/step - loss: 0.6985 - accura

In [11]:
model.save('./model_1_CNN')

INFO:tensorflow:Assets written to: ./model_1_CNN/assets
